In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
#more imports; mine, not there by default on Kaggle Kernel
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression

from sklearn.tree import DecisionTreeClassifier

from sklearn.svm import SVC

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingEnsemble


from sklearn.model_selection import validation_curve
from sklearn.model_selection import learning_curve
from sklearn.model_selection import cross_val_score

from sklearn.metrics import make_scorer
mae_scoring = make_scorer(mean_absolute_error)

from sklearn.decomposition import PCA

from sklearn.neighbors import KNeighborsClassifier

from numbers import Number

In [8]:
df = pd.read_csv("./input/train.csv")
# print columns w nas, if any
print(df.columns[pd.isnull(df).any()].tolist())
#no nas

df[:3]

[]


,id,bone_length,rotting_flesh,hair_length,has_soul,color,type
0,0,0.354512,0.350839,0.465761,0.781142,clear,Ghoul
1,1,0.575560,0.425868,0.531401,0.439899,green,Goblin
2,2,0.467875,0.354330,0.811616,0.791225,black,Ghoul


In [9]:
#dummy out color
dums = pd.get_dummies(df['color'], prefix = "col", drop_first = True )
df = pd.concat([df,dums], axis=1)
df[:10]

,id,bone_length,rotting_flesh,hair_length,has_soul,color,type,col_blood,col_blue,col_clear,col_green,col_white
0,0,0.354512,0.350839,0.465761,0.781142,clear,Ghoul,0.0,0.0,1.0,0.0,0.0
1,1,0.575560,0.425868,0.531401,0.439899,green,Goblin,0.0,0.0,0.0,1.0,0.0
2,2,0.467875,0.354330,0.811616,0.791225,black,Ghoul,0.0,0.0,0.0,0.0,0.0
3,4,0.776652,0.508723,0.636766,0.884464,black,Ghoul,0.0,0.0,0.0,0.0,0.0
4,5,0.566117,0.875862,0.418594,0.636438,green,Ghost,0.0,0.0,0.0,1.0,0.0
5,7,0.405680,0.253277,0.441420,0.280324,green,Goblin,0.0,0.0,0.0,1.0,0.0
6,8,0.399331,0.568952,0.618391,0.467901,white,Goblin,0.0,0.0,0.0,0.0,1.0
7,11,0.516224,0.536429,0.612776,0.468048,clear,Ghoul,0.0,0.0,1.0,0.0,0.0
8,12,0.314295,0.671280,0.417267,0.227548,blue,Ghost,0.0,1.0,0.0,0.0,0.0
9,19,0.280942,0.701457,0.179633,0.141183,white,Ghost,0.0,0.0,0.0,0.0,1.0


In [11]:
inputs = [col for col in df.columns if (col !='id' and col!= 'type' and col != 'color')]
y = df['type']

In [12]:
X = np.asarray(df[inputs])

In [42]:
from sklearn
#clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),n_estimators = 100)
#clf = RandomForestClassifier(n_estimators=300, max_depth = 9)
clf = SVC(kernel='sigmoid')
to_vary = 'C'
varyrange = [.01,.03,.1,.3,1,3,10,30,60,100,300,1000]
#for rbf kernel, fairly sharp peak at 10
# varyrange = range(5,50,5)
#still 10
#for sigmoid kernel, less-sharp peaks at 60 (and 300)
varyrange = range(30,150,10)
#peaks 30, 150
varyrange = range(22,42,2)
varyrange = range(110,200,10)
varyrange = range(100,1000,10)
#peak at 230
varyrange = range(200,300)
#best = tie between 232 and 233

# to_vary = 'min_impurity_split'
# varyrange = np.arange(.03,.24,.03)

# to_vary = 'max_depth'
# varyrange = range(1,15)


train_scores, test_scores = validation_curve(clf, X, y, to_vary, varyrange)
# train_sizes, train_scores, test_scores = learning_curve(clf, X, y)


train_avg = np.mean(train_scores, axis=1)
test_avg = np.mean(test_scores, axis=1)

plt.clf()
plt.plot(varyrange, train_avg, color = 'k', marker = '.')
plt.plot(varyrange, test_avg, color = 'g', marker = '.')
# plt.plot(train_sizes, train_avg, color = 'k', marker = '.')
# plt.plot(train_sizes, test_avg, color = 'g', marker = '.')

plt.show()

In [45]:
clf = SVC(kernel='sigmoid', C = 232.5)
clf.fit(X,y)
clf.score(X,y)

0.77358490566037741

In [58]:
dft = pd.read_csv("./input/test.csv")
#dummy out color
dums = pd.get_dummies(dft['color'], prefix = "col", drop_first = True )
dft = pd.concat([dft,dums], axis=1)
Xt = np.asarray(dft[inputs])

In [59]:
yp = clf.predict(Xt)
dfp = dft.copy()
dfp['class'] = yp

In [62]:
dfp.to_csv('svc2.csv', index = False, columns = ['id', 'type'])

In [61]:
dfp[:3]

,id,bone_length,rotting_flesh,hair_length,has_soul,color,col_blood,col_blue,col_clear,col_green,col_white,class
0,3,0.471774,0.387937,0.706087,0.698537,black,0.0,0.0,0.0,0.0,0.0,Ghoul
1,6,0.427332,0.645024,0.565558,0.451462,white,0.0,0.0,0.0,0.0,1.0,Goblin
2,9,0.549602,0.491931,0.660387,0.449809,black,0.0,0.0,0.0,0.0,0.0,Ghoul
